<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2022</h1>
<h1 style="font-size: 250%;">Problem Set #2 </h1>
<h1>Issued Thursday, 2/3/2022; Due Thursday, 2/17/2022</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Feel free to get help from others, but **the work you submit in should be your own.**

**Warning:** This notebook has been annotated with metadata so that it can be uploaded to the grading system. It is very important that you enter your answers in the provided cells. Answers should be placed within the cells in the indicated region (usually after a ':'). You can add extra cells to explore approaches, but only the provided cell can and will be graded. Thus, if you delete a cell and add a replacement, there is a possiblity that your problem will not be graded. If you ever need to start over, you should download a new version of the problem set and transfer your solutions to it.

In [ ]:
# Replace the following string values with the requested information
class Student:
    first = "Jenna"
    last = "Levy"
    onyen = "jenlevy"
    pid = "730280089"

You will need a the following sequence collection of 16 gene promoter regions in which you will search for <a href="http://csbio.unc.edu/mcmillan/Comp555S22/motifs.fa" download="motifs.fa">transcription binding factor motifs</a>.

The cell below provides all functions and imports necessary for this problem set. Do not import any addtional packages. Also, make sure that you ***run the following cell***.

In [1]:
import itertools
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into two lists. One of 
        headers and a second list of sequences.
        The ith index of each list correspond."""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'r')
    else:
        fp = open(filename, 'r')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

def ScanAndScoreMotif(DNA, motif):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in range(len(seq)-k+1):
            HammingDist = sum([1 for i in range(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

def MedianStringMotifSearch(DNA,k):
    """ Consider all possible 4**k motifs"""
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('ACGT', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [p for p in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

def ContainedMotifSearch(DNA,k):
    """ Consider only motifs from the given DNA sequences"""
    motifSet = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
    print("%d Motifs in our set" % len(motifSet))
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer

---
**Problem #1:** (15 points) In the cell below use the given *MedianStringMotifSearch()*, and *ScanAndScoreMotif()* functions to find an optimal 9-base motif pattern and its location in each of the 16 given promoter regions in <a href="http://csbio.unc.edu/mcmillan/Comp555S22/motifs.fa" download="motifs.fa">motifs.fa</a>. Based on this result, make a prediction of how long it would have taken to find an optimal 11-base motif (Hint: This will take some time).

In [9]:
# Read in the motifs.fa FASTA file and use its sequence 
# list as "DNA" and run the given code fragment.
headers, DNA = loadFasta("motifs.fa")
%time MedianStringMotifSearch(DNA, 9) 

Wall time: 29min 21s


([64, 29, 123, 56, 34, 93, 32, 82, 87, 120, 75, 50, 104, 26, 19, 33],
 13,
 'GACGTACAG')

**Problem #3:** (15 points) Use *ContainedMotifSearch()* to identify a candidate 11-base motif. Report the running time and minHammingDist for this solution.

In [10]:
%time ContainedMotifSearch(DNA, 11)

2190 Motifs in our set
Wall time: 7.69 s


([61, 26, 120, 53, 31, 90, 29, 79, 84, 117, 72, 47, 101, 23, 16, 30],
 26,
 'GGTGACGTACA')

**Problem #4:** (15 points) Write a variant of ContainedMotifSearch(DNA, k) called HammingMotifSearch(DNA, k) that considers all contained k-mers from the given DNA as well as all k-mers that are a Hamming distance of one from a contained k-mer. 

In [46]:
# THIS CELL WILL BE GRADED
# Your function can call the given ScanAndScoreMotif() function
# any other functions called must be defined in this cell

def HammingMotifSearch(DNA,k):
    
    motifSet = set()
    for seq in DNA:
        for i in range(len(seq)-k+1):
            motifSet.add(seq[i:i+k])
            ##enumerating through motif each time the for loop runs
            for j, base in enumerate(seq[i:i+k]):
                ##looping through every possible Nucleotide at a given base in the given kmer
                for alt in "AGCT":
                    if alt != base:
                        ##adding every kmer of a hamming distance of one to the motif set
                        motifSet.add((seq[i:i+k])[:j] + alt + (seq[i:i+k][j+1:]))
            
    print("%d Motifs in our set" % len(motifSet))
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for motif in motifSet:
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
            
    return bestAlignment, minHammingDist, kmer

%time HammingMotifSearch(DNA,11)

72940 Motifs in our set
Wall time: 5min 45s


([62, 27, 121, 54, 32, 91, 30, 80, 85, 118, 73, 48, 102, 24, 17, 31],
 16,
 'GTGACGTACAG')

**Problem #5:** (40 points) How many more k-mers are considered by *HammingMotifSearch()* than *ContainedMotifSearch()* in this specific instance? What is the upperbound of the increase in considered k-mer set sizes between these two approachs for any given data set with N contained k-mers? What is the maximum Hamming distance between any two distinct k-mers that are a Hamming distance of 1 from a given contained k-mer? Suppose that you were to modify *HammingMotifSearch()* to also include all k-mers that are a Hamming distance of 1 and 2 from each contained k-mer. How many additional k-mers might this approach consider beyond the N k-mers from the original *ContainedMotifSearch()* in the worse case? 

---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 
3. If you resubmit after the due date, you will be warned of any penalties. 
4. Problem sets will not be regraded.